## Cross-Lingual Voice Clone Demo

In [1]:
import os
import torch
from openvoice import se_extractor
from openvoice.api import ToneColorConverter

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



### Initialization

In [2]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Wed_Feb__8_05:53:42_Coordinated_Universal_Time_2023
Cuda compilation tools, release 12.1, V12.1.66
Build cuda_12.1.r12.1/compiler.32415258_0
Fri Apr 19 19:30:46 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.37                 Driver Version: 545.37       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1080      WDDM  | 00000000:06:00.0  On |                  N/A |
| 31%   52C    P0

In [3]:
print(torch.cuda.is_available())
import torch
print(torch.__version__)

True
2.1.0+cu121


In [4]:
ckpt_converter = 'checkpoints_v2/converter'
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)
output_dir = 'outputs_v2'

tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

os.makedirs(output_dir, exist_ok=True)

cuda:0


c:\Users\Ara Vardanyan\Desktop\Ara\Projects\learn-language\utils\OpenVoice\env\lib\site-packages\torch\nn\utils\weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Loaded checkpoint 'checkpoints_v2/converter/checkpoint.pth'
missing/unexpected keys: [] []


In this demo, we will use OpenAI TTS as the base speaker to produce multi-lingual speech audio. The users can flexibly change the base speaker according to their own needs. Please create a file named `.env` and place OpenAI key as `OPENAI_API_KEY=xxx`. We have also provided a Chinese base speaker model (see `demo_part1.ipynb`).

In [8]:
from openai import OpenAI
from dotenv import load_dotenv

# Please create a file named .env and place your
# OpenAI key as OPENAI_API_KEY=xxx
load_dotenv() 

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

### Obtain Tone Color Embedding

The `source_se` is the tone color embedding of the base speaker. 
It is an average for multiple sentences with multiple emotions
of the base speaker. We directly provide the result here but
the readers feel free to extract `source_se` by themselves.

In [6]:
base_speaker = f"{output_dir}/openai_source_output.wav"
source_se, audio_name = se_extractor.get_se(base_speaker, tone_color_converter, vad=True)

reference_speaker = 'resources/andres-clip-2.mp3' # This is the voice you want to clone
target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, vad=True)

OpenVoice version: v2
[(0.0, 50.496)]
after vad: dur = 50.496


c:\Users\Ara Vardanyan\Desktop\Ara\Projects\learn-language\utils\OpenVoice\env\lib\site-packages\torch\functional.py:650: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\SpectralOps.cpp:868.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


OpenVoice version: v2
[(0.0, 28.499625)]
after vad: dur = 28.498979591836736


### Inference

In [12]:
import time

# Run the base speaker tts
text = [
    "MyShell is a decentralized and comprehensive platform for discovering, creating, and staking AI-native apps.",
    "MyShell es una plataforma descentralizada y completa para descubrir, crear y apostar por aplicaciones nativas de IA.",
    "¡Hola! Mi nombre es Andrés! I am here to chat and help you practice your Spanish! ¡Yo puedo hablar en español, and I can speak in English as well! ¿Y tu, cómo te llamas?",
    # "MyShell est une plateforme décentralisée et complète pour découvrir, créer et miser sur des applications natives d'IA.",
    # "MyShell ist eine dezentralisierte und umfassende Plattform zum Entdecken, Erstellen und Staken von KI-nativen Apps.",
    # "MyShell è una piattaforma decentralizzata e completa per scoprire, creare e scommettere su app native di intelligenza artificiale.",
    # "MyShellは、AIネイティブアプリの発見、作成、およびステーキングのための分散型かつ包括的なプラットフォームです。",
    # "MyShell — это децентрализованная и всеобъемлющая платформа для обнаружения, создания и стейкинга AI-ориентированных приложений.",
    # "MyShell هي منصة لامركزية وشاملة لاكتشاف وإنشاء ورهان تطبيقات الذكاء الاصطناعي الأصلية.",
    # "MyShell是一个去中心化且全面的平台，用于发现、创建和投资AI原生应用程序。",
    # "MyShell एक विकेंद्रीकृत और व्यापक मंच है, जो AI-मूल ऐप्स की खोज, सृजन और स्टेकिंग के लिए है।",
    # "MyShell é uma plataforma descentralizada e abrangente para descobrir, criar e apostar em aplicativos nativos de IA."
]
src_path = f'{output_dir}/tmp.wav'

for i, t in enumerate(text):

    start_time = time.time()
    
    response = client.audio.speech.create(
        model="tts-1",
        voice="nova",
        input=t,
    )

    response.stream_to_file(src_path)

    end_time = time.time()

    time_tts = end_time - start_time

    print(f"Time taken to generate tts: {time_tts} seconds")

    save_path = f'{output_dir}/output_crosslingual_{i}.wav'

    start_time = time.time()

    # Run the tone color converter
    encode_message = "@MyShell"
    tone_color_converter.convert(
        audio_src_path=src_path, 
        src_se=source_se, 
        tgt_se=target_se, 
        output_path=save_path,
        message=encode_message)

    end_time = time.time()

    time_color_converter = end_time - start_time
    print(f"Time taken to convert tone color: {time_color_converter} seconds")

    print(f"Total time taken: {time_tts + time_color_converter} seconds")

C:\Users\Ara Vardanyan\AppData\Local\Temp\ipykernel_9276\3111384474.py:30: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(src_path)


Time taken to generate tts: 1.9779090881347656 seconds
Time taken to convert tone color: 0.5071702003479004 seconds
Total time taken: 2.485079288482666 seconds
Time taken to generate tts: 2.022409439086914 seconds
Time taken to convert tone color: 0.48197102546691895 seconds
Total time taken: 2.504380464553833 seconds
Time taken to generate tts: 2.656018018722534 seconds
Time taken to convert tone color: 1.274646282196045 seconds
Total time taken: 3.930664300918579 seconds
